Numerical analysis (or data analysis) is what Python is especially good at.  And it's possibly why you want to learn Python?  Now we will use a simple project to guide you.  Let's use Python to play music.

In [ ]:
# This cell sets up helpers for demonstration in the course.
# Before the end of this session, you shouldn't take this as examples.

%matplotlib inline

# Import necessary modules.

import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Audio, HTML

# Define constants.

sampling_freq = 44100 # 44.1 kHz.

# Define demonstrative helper functions.

def time_data(duration, rate=sampling_freq):
    return np.linspace(0, duration, num=int(rate*duration))

def sine_data(freq, time, damp=0):
    data = 2**13 * np.sin(2*np.pi * freq * time)
    if damp: # Remove gitters.
        todamp = int(damp * sampling_freq)
        darr = np.arange(todamp)/(todamp-1)
        darr = np.sin(darr * np.pi / 2)
        data[:todamp] *= darr
        data[-todamp:] *= 1-darr
    return data

def note_freq(note):
    """Parse note symbol "a4", "c4", etc. and calculate frequency (in Hz)."""
    if ' ' == note:
        return 0.0
    f0 = 440.0 # a4
    note = note.lower()
    diff = {'a': 0, 'b': 2, 'c': -9, 'd': -7, 'e': -5, 'f': -4, 'g': -2}[note[0]]
    diff += {'#': 1, 'b': -1}.get(note[1], 0)
    diff += 12 * (int(note[-1])-4)
    return f0 * 2 ** (float(diff)/12)

def play_notes(duration, *notes):
    time = time_data(duration, sampling_freq)
    alist = [sine_data(note_freq(n), time) for n in notes]
    data = np.vstack(alist).sum(axis=0)
    display(Audio(data.astype('int16'), rate=sampling_freq))
    return time, data

def play_numbered(numbered_notation, bpm=90, base_octave=4):
    # Middle C is designated C4
    note_map = {'0':' ', '1':'c', '2':'d', '3':'e', '4':'f', '5':'g', '6':'a', '7':'b'}
    opt_symbols = (
        # octaves
        '^', 'v',
        # accidentals
        '#', 'b',
        # note length
        '-', '.', '_',
        # misc
        '|', ' ', ','
    )
    beat_sec = 60.0 / bpm
    signals = []
    # init numbered
    default_num = lambda: (base_octave, '', 1.0)
    octave, accidental, nbeat = default_num()
    for it, num in enumerate(numbered_notation):
        # raise/lower octave
        if num == '^':
            octave += 1
        elif num == 'v':
            octave -= 1
        
        # accidentals
        if num in '#b':
            accidental = num
        
        if num in opt_symbols:
            continue
            
        for ahead in numbered_notation[it+1:]:
            if '-' == ahead:
                nbeat += 1
            elif '.' == ahead:
                nbeat += 0.5
            elif '_' == ahead:
                nbeat /= 2
            else:
                break
        
        time = time_data(beat_sec * nbeat, sampling_freq)
        note = note_map[num]
        if note != ' ':
            note = note + accidental + str(octave)
        freq = note_freq(note)
        signal = sine_data(freq, time, damp=beat_sec*0.1)
        signals.append(signal)
        
        # reset to default
        octave, accidental, nbeat = default_num()
        
    data = np.hstack(signals)
    display(Audio(data.astype('int16'), rate=sampling_freq))

Run the following cells.  They'll show you what the above code do.

In [ ]:
print("Play the C chord:")
time_point, data = play_notes(3, 'c4', 'e4', 'g4')

print("Plot the first 1000 time points (around %.0f ms):" % (1.e6/sampling_freq))
_ = plt.plot(time_point[0:1000], data[0:1000])

In [ ]:
print("Twinkle Twinkle Little Star:")
play_numbered('1155|665-|4433|221-|5544|332-|5544|332-|1155|665-|4433|221-')

print("Hänschen Klein:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0')

print("Hänschen Klein at 120 BPM:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0', bpm=120)

print("Marshmello - TELL ME as 142 BPM:")
play_numbered('#5#1-#5|4-#1_#2_4|#5#1-^#1|^1-#5_#6_4' * 2, bpm=142)

Now, step by step, we'll show you how we made Python sing.

# Variables

Variables are the things that Python uses to keep track of information.  Let's see an example:

In [ ]:
# First, we create some data by calling the helper function "time_data()".
# At the same time, we name it the "time", which is the variable.
time = time_data(10)
# By using the "time" varaible, we can send the data to the tool "plt.plot()",
# and plot the data.  As they are shown, the data are a series of increasing
# values of the same difference.
plt.plot(time)

The example demonstrates a key feature of a variable: It is a handle to access the associated data.  The action (line 3) use the variable "`time`" to name the data returned by `time_data()`, so that we can tell Python to send the data to `plt.plot()`.  Variables give us the ability to name things in Python.  This is a very basic and critical thing in programming.

As an exercise, try to do a similar thing but use another variable.  You should still see a straight line, but the maximum value becomes 12 instead of 10.

In [ ]:
other_time = time_data(12)
plt.plot(other_time)

In Python, a variable can be any type of data.  You don't need to specify or know the kind of data before using the variable.  But you do need to have the variable set before use it.

In [ ]:
a_number = 10
print(a_number)
a_word = "advancement"
print(a_word)

print(a_condition) # Won't work since a_condition isn't yet set.
a_condition

## Numbers and Strings

Numbers and strings are the most straight-forward variables.  By using the high-school algebra we immedinately make sense of the following examples.

In [ ]:
interest_rate = 0.02 # 2% compound interest rate (annualized).
principal = 1_000_000 # The money you save monthly.
interest = principal*(1+interest_rate/12)**12 - principal # The interest rolls in monthly.

print("Interest is {:,.0f} with {:,.0f} at rate {:.1%}.".format(interest, principal, interest_rate))

Say the annual interest rate is 2%.  Although Python cannot allow a percentage number, we know it is simply the real number `0.02`, and write it as

```python
interest_rate = 0.02 # 2% compound interest rate (annualized).
```

In Python, everythin that follows the "`#`" (pronounced "sharp" or "dial") symbol is taken as comments (code remarks).  They are considered not part of the code, and allow programmers to use natural language to explain the code.  You will expect a lot of them in this notebook.

For numbers, modern Python has a neat feature, is to allow you to insert eye candy for better grouping.  In the second line the underscore (`_`) separates the number 1 million by thousand:

```python
principal = 1_000_000
```

Compare to the representation without the underscores:

```python
principal = 1000000
```

The former helps you avoid error.

The named variables allow us to write the formula in the third line:

```python
interest = saving*(1+interest_rate/12)**12 - saving # The interest rolls in monthly.
```

The bank uses a compound interest rate, and we want to know how much money we eventually get in the whole year.  The interest is evaluated every month, so the rate needs to be divided by 12.  It's the compound rate, so we multiply the principal by `(1+interest_rate/12)` 12 times.

At the end we show the results using the built-in `print()` function:

```python
print("Interest is {:,.0f} with {:,.0f} at rate {:.1%}.".format(interest, principal, interest_rate))
```

The first part passed into `print()` is the template string (also called formatting string):

```python
"Interest is {:,.0f} with {:,.0f} at rate {:.1%}."
```

The second part uses a method attached to the template string to fill in the data:

```python
.format(interest, principal, interest_rate))
```

In the end, Jupyter shows the formatted string:

```
Interest is 20,184 with 1,000,000 at rate 2.0%.
```

The substring enclosed by the pair of curled braces (`{}`) in the template string specifies how the associated variable is formatted.  We use two of them in the above example:

* `{:,.0f}` says to format the real number using fixed-point notation, use `,` to group by thousand, and show no (0) digit after decimal points.
* `{:.1%}` says to format the real number as percentage, and show 1 digit after decimal point.

But if you don't bother the detailed formatting, it's fine to simply write `{}` as placeholders.  The result is only slightly less clear but you still get it.

In [ ]:
print("Interest is {} with {} at rate {}.".format(interest, principal, interest_rate))

## Values and Containers

In Python, variables can be broadly categorized into two kinds: a mere value and a container of other values or containers.

# Tables and Arrays

# Present Data